# imports

In [1]:
!pip install einops
!pip install zarr
!pip install xarray[io]
!pip install -Uqq ipdb
from numpy import save, load
from pathlib import Path
import dask.array as da
import warnings
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import ipdb
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, roc_auc_score, average_precision_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import math
from torch.optim.optimizer import Optimizer
import pandas as pd
from einops import rearrange
from torch.nn import functional as F
import xarray as xr
from torch.utils.data import Dataset, DataLoader
import zarr
import sys


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.0 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5050 sha256=cb22c4277b94721cd26cadf27d04ebeded6a2747b72ecc59f17e1b8383fe693e
  Stored in directory: /root/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built asciitree
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.4 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%pdb off

Automatic pdb calling has been turned OFF


# Dataset + Dataloader

In [3]:
class TemporalDatasetFromNumpy(Dataset):
    def __init__(self, numpy_array, sequence_length, train_ratio, pin_memory):
        self.numpy_array = numpy_array #shape (40, 40, 918, 29), lat, lon, time, var
        self.sequence_length = sequence_length
        self.pin_memory = pin_memory
        
        self.lat_size, self.lon_size, self.time_size, self.num_variables = self.numpy_array.shape

        self.time_size -= (sequence_length) #num of timesteps with targets
        self.numpy_array = self.numpy_array[:self.time_size]
        # 40, 40, 854, 29
        # 0 to 39
        
        self.train_ratio = train_ratio
        
        self.pos_indices = np.column_stack(np.where(self.numpy_array[:,:,self.sequence_length:, -1] == 1)) 
        # shape (number_of_occurences) by (lat_indices, lon_indices, time_indices)
        self.neg_indices = np.column_stack(np.where(self.numpy_array[:,:,self.sequence_length:, -1] == 0))
            
        #ipdb.set_trace()
        self.total_indices = None
        
        if (self.train_ratio): # True -> training; False -> eval
            
            neg_to_pos_ratio = 2
            num_rows = len(self.pos_indices)*neg_to_pos_ratio
            random_rows = np.random.choice(self.neg_indices.shape[0], size=num_rows, replace=False)
            #ipdb.set_trace()
            neg_indices_subset = self.neg_indices[random_rows, :]
            #ipdb.set_trace()
            
            self.total_indices = np.concatenate((self.pos_indices, neg_indices_subset), axis=0)
            #self.total_indices = np.concatenate((self.pos_indices, self.neg_indices), axis=0)
            
        else:
            self.total_indices = np.concatenate((self.pos_indices, self.neg_indices), axis=0)
        
        np.random.shuffle(self.total_indices)
        #ipdb.set_trace()
            
    def __len__(self): # num_samples * num_timesteps
        return len(self.total_indices)

    def __getitem__(self, index): # np array shape (40, 40, 918, 29), lat, lon, time, var
        #total indices shape (num samples) by (lat_indices, lon_indices, time_indices)
        lat_index = self.total_indices[index, 0]
        lon_index = self.total_indices[index, 1]
        time_index = self.total_indices[index, 2]

        # ex timesteps 0 to 64: 65 timesteps in total, features takes 0 to 63, target is 64th timestep
        target_np = self.numpy_array[lat_index, lon_index, time_index+sequence_length, -1] # only last variable gwis_ba
        features_np = self.numpy_array[lat_index, lon_index, time_index:time_index+sequence_length, : ] # all variables
        # lat, lon, time, var
        
        
        return features_np.squeeze(), target_np.squeeze()

def temporal_dataloader(numpy_array, sequence_length, batch_size, train_ratio, pin_memory=False, num_workers=0):
    temporal_dataset = TemporalDatasetFromNumpy(numpy_array, sequence_length, train_ratio, pin_memory)
    dataloader = DataLoader(temporal_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False)
    return dataloader


# Trainer + Evaluator

In [4]:
# Define the training loop
def train(model, dataloader): #each dataloader gives 1203*150 samples
    total_loss = 0.0
    total_samples = 0.0

    for batch_features, batch_target in tqdm(dataloader, desc = 'training'): #ndvi missing values
        #ipdb.set_trace()
        batch_features = batch_features.numpy()
        batch_target = batch_target.numpy()
        shape = batch_features.shape
        batch_features_reshaped = batch_features.reshape(shape[0],-1)
        #Drop all rows containing any nan:
        #ipdb.set_trace()
        mask = ~np.isnan(batch_features_reshaped).any(axis = 1)
        batch_features_reshaped = batch_features_reshaped[mask]
        #Reshape back:
        #ipdb.set_trace()
        batch_features = batch_features_reshaped#.reshape(batch_features_reshaped.shape[0],*shape[1:])
        batch_target = batch_target[mask].squeeze()
        #ipdb.set_trace()
            
        model.fit(batch_features, batch_target)
        model.n_estimators += 1

def evaluate_model(model, dataloader):
    
    total_f1 = 0.0
    total_recall = 0.0
    total_precision = 0.0
    total_accuracy = 0.0
    total_aucroc = 0.0
    total_auprc = 0.0
    total_samples = 0.0 #because each batch is the same size

    for inputs, targets in tqdm(dataloader, desc = 'evaluating'):
        inputs = inputs.numpy()
        targets = targets.numpy()
        shape = inputs.shape
        inputs_reshaped = inputs.reshape(shape[0],-1)
        #Drop all rows containing any nan:
        mask = ~np.isnan(inputs_reshaped).any(axis = 1)
        inputs_reshaped = inputs_reshaped[mask]
        #ipdb.set_trace()
        inputs = inputs_reshaped#.reshape(inputs_reshaped.shape[0],*shape[1:])
        targets = targets[mask].squeeze()

        outputs = model.predict(inputs)
        
        num_samples = len(targets)

        total_f1 += f1_score(targets, outputs, pos_label=1) * num_samples
        total_precision += precision_score(targets, outputs, pos_label=1) * num_samples
        total_accuracy += accuracy_score(targets, outputs) * num_samples
        total_recall += recall_score(targets, outputs, pos_label=1) * num_samples
        total_aucroc += roc_auc_score(targets, outputs) * num_samples
        total_auprc += average_precision_score(targets, outputs, pos_label=1) * num_samples
        total_samples += num_samples

    #average_f1 = total_f1 / total_batches
    #average_recall = total_recall / total_batches
    #average_precision = total_precision / total_batches
    #average_aucroc = total_aucroc / total_batches
    #average_accuracy = total_accuracy / total_batches
    return total_f1, total_recall, total_precision, total_auprc, total_aucroc, total_accuracy, total_samples

# RUN!!!

In [5]:
batch_size = 100
file_path = Path("/kaggle/input/california-spatial-temporal-fire-dataset/numpy_california_spatialtemporal_dataset.npy")
numpy_array = np.load(file_path)[:,:,:700,:]
model = RandomForestClassifier(n_estimators=100, warm_start = True)
sequence_length = 64
dataloader = temporal_dataloader(numpy_array, sequence_length, batch_size, train_ratio = True, pin_memory = False, num_workers=0)
train(model, dataloader)
print('done training')

training: 100%|██████████| 193/193 [00:01<00:00, 127.05it/s]

done training


In [6]:
test_dataloader = temporal_dataloader(np.load(file_path)[:,:,700:,:], sequence_length, batch_size, train_ratio = True, pin_memory = False, num_workers=0)
total_f1, total_recall, total_precision, total_auprc, total_aucroc, total_accuracy, total_samples = evaluate_model(model, dataloader)
print(total_f1 / total_samples)
print(total_recall / total_samples)
print(total_precision / total_samples)
print(total_auprc / total_samples)
print(total_aucroc / total_samples)
print(total_accuracy / total_samples)
print(total_samples)

evaluating: 100%|██████████| 193/193 [00:08<00:00, 22.53it/s]

0.4747256177105379
0.3753840871251796
0.6616192659416837
0.4632118727115706
0.6399501963913266
0.7284315249494793
19299.0
